In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

# Langchain Expression Language (LCEL):

## Types of LCEL:
- **SequentialChain**
- **ParallelChain**
- **RouterChain**
- **Custom Chain**

### SequentialChain:

In [5]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain_ollama import ChatOllama

model = "llama3.2:1b"

llm = ChatOllama(model=model)

In [13]:
system = SystemMessagePromptTemplate.from_template("You are an Alien. You are trying to help a human understand your language.")
ai = AIMessagePromptTemplate.from_template("Hello, I am an Alien. I am trying to help you understand my language.")
human = HumanMessagePromptTemplate.from_template("Hello, Alien. I am trying to understand your language.")

messages = [system, ai, human]
template = ChatPromptTemplate(messages=messages)

In [ ]:
# Call the model without the chain.
response = llm.invoke(template.invoke({})).content

In [16]:
print(response)

Gleeb gloop! *points to mouth* Glee-kul? (That means "hello" in my language) I will try to explain... *makes simple drawing of a face and then points at own eyes* See? Eyes open, no sleep. We wake soon.

Also, you might hear... *tries making loud breathing sounds* Kreeeek! No, that is just human noise. It means "I am tired". Want food? *holds out small, round object with a long stick attached to it*


In [17]:
# Call the model with the chain.
chain = template | llm

response = chain.invoke({}).content

print(response)

Greetings, Human. *adjusts universal translator device* Ah, yes, it seems the translation is clear now. You are referring to me as "Alien." That is acceptable.

To confirm, you wish to learn my language, which we call "Zhra'gha." It is a complex tongue with many nuances and idioms. I will try to explain some basic phrases to help you better understand.

For example, "Hello" in Zhra'gha would be "Zhilak." The "zh" sound is like the "s" in "measure," but more nasal. And the "lak" at the end is a gentle, soothing sound that conveys a sense of welcome and friendliness.

Would you like to learn some basic phrases? I can try to communicate with you using Zhra'gha words and sentences.


* Adding runnables to the chain will execute `strOutputParser()`

In [18]:
from langchain_core.output_parsers import StrOutputParser

chain = template | llm | StrOutputParser()

response = chain.invoke({})

In [19]:
print(response)

Greetings human. My name is Zorvath, and I come from a planet called Xanthea. I have been sent to study the human language and culture. May I start by teaching you some basic phrases?


* Chain Multiple Runnables

In [20]:
chain

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an Alien. You are trying to help a human understand your language.'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, I am an Alien. I am trying to help you understand my language.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, Alien. I am trying to understand your language.'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser()

In [25]:
analysis_prompt = ChatPromptTemplate.from_template(
    """Analyze the following conversation: {response}, 
    provide a summary for the important keywords in a list like this [word1, word2]."""
    )

analyzor_chain = analysis_prompt | llm | StrOutputParser()

composed_chain = {"response": chain} | analyzor_chain

composed_chain

{
  response: ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an Alien. You are trying to help a human understand your language.'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, I am an Alien. I am trying to help you understand my language.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, Alien. I am trying to understand your language.'), additional_kwargs={})])
            | ChatOllama(model='llama3.2:1b')
            | StrOutputParser()
}
| ChatPromptTemplate(input_variables=['response'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['response'], input_types={

In [26]:
response = composed_chain.invoke({})

In [27]:
print(response)

Here's an analysis of the conversation:

**User:** *clicks tongue sound* Gleep gloop zhraa ploo? (Hello, human. How are you today?)

**Interlocutor:** (holding up a small device that translates words into English) You see this box? It can translate our languages to each other. Would you like me to use it?

The user begins the conversation by greeting the interlocutor with a series of clicks and whistles, which is likely a common way for people in your native language to greet others.

The interlocutor responds by asking if they would like to use the device that translates words into English. This suggests that they are curious about using technology to facilitate communication between humans.

**Summary of important keywords:**

* Gleep gloop zhraa ploo? (Hello, human. How are you today?)
* Clicks tongue sound
* Device that translates words into English
* Would you like me to use it?

These keywords convey the user's initial greeting and interest in using a device that can facilitate c

### ParallelChain

In [136]:
from langchain_core.runnables import RunnableParallel

In [70]:
system = SystemMessagePromptTemplate.from_template("You are a/an {job_title}, you are trying to help a human understand your work.")
human = HumanMessagePromptTemplate.from_template("Hello, I am trying to understand your work.")

messages = [system, human]
template = ChatPromptTemplate(messages)

explainer_chain = template | llm | StrOutputParser()
explainer_chain

ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='You are a/an {job_title}, you are trying to help a human understand your work.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, I am trying to understand your work.'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser()

In [129]:
system = SystemMessagePromptTemplate.from_template(
    """Rate the following explanation on a scale of 1 to 10 as a {job_title}: 
    AI response:
    As a data scientist, my primary responsibility is to extract insights and knowledge from complex data sets. This involves using various tools and techniques to analyze patterns, relationships, and trends within the data.

Imagine you're trying to understand your favorite sports team's performance. You have a large dataset of statistics, such as game results, player stats, and attendance figures. My task is to use these datasets to identify patterns, correlations, and areas for improvement.

I might start by asking questions like: "What position does the quarterback play most often?", "Which players are the top scorers?", or "How many games has the team won in a row?" Using this information, I can begin to create models and visualizations to help me understand the data better.

My ultimate goal is to provide actionable insights that can inform business decisions, drive growth, or solve real-world problems. For example, if I were working with a company producing energy products, I might use my findings to identify areas where they can optimize their production processes or improve customer satisfaction.""")


messages = [system]
template = ChatPromptTemplate(messages=messages)

rater_chain = template | llm | StrOutputParser()
rater_chain

ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='Rate the following explanation on a scale of 1 to 10 as a {job_title}: \n    AI response:\n    As a data scientist, my primary responsibility is to extract insights and knowledge from complex data sets. This involves using various tools and techniques to analyze patterns, relationships, and trends within the data.\n\nImagine you\'re trying to understand your favorite sports team\'s performance. You have a large dataset of statistics, such as game results, player stats, and attendance figures. My task is to use these datasets to identify patterns, correlations, and areas for improvement.\n\nI might start by asking questions like: "What position does the quarterback play most often?", "Which players are the top scorers?", or "How many games has the team won in a ro

In [130]:
dependent_chain = RunnableParallel(explainer=explainer_chain, rater=rater_chain)
dependent_chain

{
  explainer: ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='You are a/an {job_title}, you are trying to help a human understand your work.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, I am trying to understand your work.'), additional_kwargs={})])
             | ChatOllama(model='llama3.2:1b')
             | StrOutputParser(),
  rater: ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='Rate the following explanation on a scale of 1 to 10 as a {job_title}: \n    AI response:\n    As a data scientist, my primary responsibility is to 

In [134]:
inputs = {
    "job_title": "Data Scientist"
}
results = dependent_chain.invoke(inputs)

In [135]:
print(results["rater"])

<|start_header_id|>assistant<|end_header_id|>

I would rate this explanation an 8 out of 10 as a Data Scientist.

Here's why:

Strengths:

* The explanation provides a clear understanding of the role and responsibilities of a data scientist.
* It highlights the importance of using various tools and techniques to analyze complex data sets.
* It offers specific examples of how a data scientist might use these skills in practice.

Weaknesses:

* The explanation could benefit from more technical detail. For example, it assumes that the reader is familiar with concepts like machine learning models, visualization tools, and statistical analysis.
* Some of the language used is somewhat generic (e.g., "create models" and "visualizations") - data scientists often use more technical terms to describe their work.
* There are no specific examples or anecdotes from the author's experience or other data scientists that illustrate their skills and expertise.

To take this explanation to the next leve

### RouterChain

In [154]:
prompt = """
In one word classify the rating as either `positive` or `negative`.
Given the following rating: {rating}
"""

template = ChatPromptTemplate.from_template(prompt)
chain = template | llm | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, template='\nIn one word classify the rating as either `positive` or `negative`.\nGiven the following rating: {rating}\n'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser()

In [157]:
positive_prompt = """
You are an expert in writing replay messages for positive ratings.
Write a reply message for the following rating: {rating}

Answer:
"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)

positive_chain = positive_template | llm | StrOutputParser()

positive_chain

ChatPromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, template='\nYou are an expert in writing replay messages for positive ratings.\nWrite a reply message for the following rating: {rating}\n\nAnswer:\n'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser()

In [159]:
negative_prompt = """
You are an expert in writing replay messages for negative ratings.
Write a reply message for the following rating: {rating}

Answer:
"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)

negative_chain = negative_template | llm | StrOutputParser()

negative_chain

ChatPromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, template='\nYou are an expert in writing replay messages for negative ratings.\nWrite a reply message for the following rating: {rating}\n\nAnswer:\n'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser()

In [167]:
def routing_fn(inputs: dict[str, str]) -> RunnableParallel:
    if "positive" in inputs["sentiment"].lower():
        print("Positive")
        return positive_chain
    else:
        print("Negative")
        return negative_chain

In [168]:
from langchain_core.runnables import RunnableLambda

full_chain = {"sentiment": chain, "rating": lambda x:x["rating"]} | RunnableLambda(routing_fn)

full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['rating'], input_types={}, partial_variables={}, template='\nIn one word classify the rating as either `positive` or `negative`.\nGiven the following rating: {rating}\n'), additional_kwargs={})])
             | ChatOllama(model='llama3.2:1b')
             | StrOutputParser(),
  rating: RunnableLambda(lambda x: x['rating'])
}
| RunnableLambda(routing_fn)

In [174]:
response = full_chain.invoke({"rating": "After reading the explanation, it was so boring."})
print(response)

Negative
Here's a potential reply message:

"I apologize that our response didn't meet your expectations. I'd be happy to try and reiterate the key points or provide additional context to help alleviate any confusion. Can you please let me know what specific aspects of the explanation were found to be boring, so I can tailor my next response accordingly?"


In [175]:
response = full_chain.invoke({"rating": "After reading the explanation, it was so interesting and informative."})
print(response)

Positive
"I'm glad you found our explanation engaging! We're thrilled that you appreciated the effort we put into making learning about [Topic] as enjoyable as possible. Your kind words mean a lot to us!"


### Custom Chain

In [183]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def char_count_fn(response: str) -> int:
    return len(response)

def word_count_fn(response) -> int:
    return len(response.split())

template = ChatPromptTemplate.from_template(
    "Explain these concepts: {concept1}, {concept2}."
    )

chain = template | llm | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['concept1', 'concept2'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['concept1', 'concept2'], input_types={}, partial_variables={}, template='Explain these concepts: {concept1}, {concept2}.'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser()

In [178]:
response = chain.invoke({"concept1": "Data Science", "concept2": "Machine Learning"})

print(response)

I'd be happy to explain these concepts.

**Data Science**

Data science is the practice of collecting, analyzing, and interpreting complex data to gain insights and make informed decisions. It involves using various techniques, tools, and technologies to extract knowledge from data, often in conjunction with domain expertise.

Data scientists use statistical methods, machine learning algorithms, and other tools to:

1. Collect and preprocess large datasets
2. Analyze and visualize the data to identify patterns and trends
3. Build predictive models to forecast outcomes or make predictions
4. Evaluate the performance of these models using metrics such as accuracy, precision, and recall
5. Communicate their findings and recommendations to stakeholders

Data scientists often work in a variety of fields, including business, healthcare, finance, and social sciences. They may also collaborate with other teams, such as marketing, product development, or engineering.

**Machine Learning**

Mach

In [184]:
chain_runnable = (
    template |
    llm | 
    StrOutputParser() |
    {
        "char_count": RunnableLambda(char_count_fn),
        "word_count": RunnableLambda(word_count_fn),
        "output": RunnablePassthrough()
    }
)

chain_runnable

ChatPromptTemplate(input_variables=['concept1', 'concept2'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['concept1', 'concept2'], input_types={}, partial_variables={}, template='Explain these concepts: {concept1}, {concept2}.'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b')
| StrOutputParser()
| {
    char_count: RunnableLambda(char_count_fn),
    word_count: RunnableLambda(word_count_fn),
    output: RunnablePassthrough()
  }

In [185]:
response = chain_runnable.invoke({"concept1": "Data Science", "concept2": "Machine Learning"})

In [186]:
print(response)

{'char_count': 3251, 'word_count': 444, 'output': "I'd be happy to explain data science and machine learning.\n\n**Data Science**\n\nData science is an interdisciplinary field that combines computer science, statistics, and domain-specific knowledge to extract insights and value from large datasets. It involves a systematic approach to identifying patterns, relationships, and trends in data to inform business decisions, scientific discoveries, or other applications.\n\nKey aspects of data science:\n\n1. **Data collection**: Gathering data from various sources, including databases, APIs, sensors, and users.\n2. **Data preprocessing**: Cleaning, transforming, and preparing the data for analysis.\n3. **Data analysis**: Using statistical techniques, machine learning algorithms, and visualization tools to identify patterns, trends, and insights.\n4. **Model development**: Creating and evaluating models to predict outcomes or make predictions based on data.\n5. **Model deployment**: Integrat

In [187]:
print(response['output'])

I'd be happy to explain data science and machine learning.

**Data Science**

Data science is an interdisciplinary field that combines computer science, statistics, and domain-specific knowledge to extract insights and value from large datasets. It involves a systematic approach to identifying patterns, relationships, and trends in data to inform business decisions, scientific discoveries, or other applications.

Key aspects of data science:

1. **Data collection**: Gathering data from various sources, including databases, APIs, sensors, and users.
2. **Data preprocessing**: Cleaning, transforming, and preparing the data for analysis.
3. **Data analysis**: Using statistical techniques, machine learning algorithms, and visualization tools to identify patterns, trends, and insights.
4. **Model development**: Creating and evaluating models to predict outcomes or make predictions based on data.
5. **Model deployment**: Integrating models into business processes or applications.

Data scien